In [ ]:
import json
import argparse
import cv2 as cv
import os
import numpy as np
from pathlib import Path
from pycocotools import mask as mask_utils
import csv
from segmentation_labels_coco import get_image_and_mask, generate_annotations_for_image, filter_cells_by_area, crop_image_from_annotation, transform_crop, save_cropped_images, save_json_and_csv, transform_images_to_white

In [ ]:
# Insert the path to the input, target and output folders (required)
input_path = ""
target_path = ""
output_path = ""

# Set the parameters for the segmentation
padding = 15
threshold = 50
save_reviewed = True

annotations = []
annotation_id = 1

for image_id, image_file in enumerate(input_path.iterdir(), start=1):
    if image_file.suffix not in [".png", ".jpg", ".jpeg"]:
        continue

    mask_file = target_path / image_file.name
    if not mask_file.exists():
        continue

    image, mask = get_image_and_mask(image_file, mask_file)

    image_annotations, annotation_id = generate_annotations_for_image(
        image, mask, image_file, image_id, annotation_id, padding
    )

    annotations.extend(image_annotations)

In [ ]:
annotations = filter_cells_by_area(annotations, threshold)

In [ ]:
annotations_processed = []

for annotation in annotations:
    cropped_mask, _, target_crop_name = crop_image_from_annotation(annotation)
    _, review_required = transform_crop(cropped_mask, target_crop_name)
    if review_required and not save_reviewed:
        continue
    annotation['review_required'] = review_required
    annotations_processed.append(annotation)

In [ ]:
for annotation in annotations_processed:
    cropped_mask, cropped_image, target_crop_name = crop_image_from_annotation(annotation)
    save_cropped_images(output_path, cropped_mask, cropped_image, target_crop_name)

In [ ]:
save_json_and_csv(annotations_processed, output_path)

In [ ]:
transform_images_to_white(output_path / "target")